# Begin model training

## Load libraries

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import os

## Setup Colab

In [2]:
COLAB = 'google.colab' in str(get_ipython())

if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT = '/content/drive/MyDrive/fyp/collabs/'
else:
    ROOT = os.path.join(os.getcwd(), '..', '..') 

## Load Colab libraries

In [3]:
if COLAB:
    !pip install pickle5
    import pickle5 as pickle

## Load custom modules

In [4]:
from nnssa.constants import *
from nnssa.evaluate import evaluate

## Set random seed for reproducability

In [5]:
SEED = 42
np.random.seed(SEED)

## Load TensorFlow modules

In [6]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [7]:
def get_metrics():
    return [
        tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.15),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]

## Enable TPU

In [8]:
if ('COLAB_TPU_ADDR' in os.environ.keys()):
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_host(resolver.master())
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

## Progress bars!

In [22]:
from tqdm import tqdm
tqdm.pandas()

/Users/admin/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Load dataset

In [101]:
if COLAB:
    salami_beats = pickle.load(open(os.path.join(ROOT, SUB_DIVS_DIR, 'bars', 'salami.p'), 'rb'))
else:
    salami_beats = pd.read_pickle(os.path.join(ROOT, SUB_DIVS_DIR, 'bars', 'salami.p'))
salami_beats = salami_beats.head(100)
salami_beats.head()

,File,Sub_Divisions,Binary_Labels,Weighted_Labels,Weights,IDS,Beat_times,Labels
0,956,/Users/admin/Downloads/fypdataset/notebooks/05...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.5, 3.0, 0.5, 1.0, 1.0, ...","[956, 956, 956, 956, 956, 956, 956, 956, 956, ...","[0.0, 2.5541950113378684, 3.9009523809523814, ...","[28.746303854, 49.357959183, 91.03056689299999..."
1,958,/Users/admin/Downloads/fypdataset/notebooks/05...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, ...","[3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, ...","[958, 958, 958, 958, 958, 958, 958, 958, 958, ...","[0.0, 0.18575963718820865, 0.8823582766439909,...","[0.045, 26.129208333, 71.4750625, 116.7300625,..."
2,960,/Users/admin/Downloads/fypdataset/notebooks/05...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[960, 960, 960, 960, 960, 960, 960, 960, 960, ...","[0.0, 0.18575963718820865, 1.3931972789115643,...","[0.048979590999999996, 76.260068027, 108.70204..."
3,962,/Users/admin/Downloads/fypdataset/notebooks/05...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 3.0, 0.5, 1.0, ...","[962, 962, 962, 962, 962, 962, 962, 962, 962, ...","[0.0, 0.3715192743764173, 1.0216780045351477, ...","[15.18585034, 56.47031746, 104.25839002200001,..."
6,968,/Users/admin/Downloads/fypdataset/notebooks/05...,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 0.0, 2.0, ...","[3.0, 3.0, 0.5, 1.0, 0.5, 3.0, 0.5, 1.0, 0.5, ...","[968, 968, 968, 968, 968, 968, 968, 968, 968, ...","[0.0, 0.4643990929705216, 1.7182766439909296, ...","[0.865306122, 6.802834467, 27.484149659, 47.89..."


## Load Mel Spectrograms

In [102]:
def load_np(file):
    return np.load(os.path.join(ROOT, SUB_DIVS_DIR, file), allow_pickle=True)

In [103]:
salami_beats['Sub_Divisions'] = salami_beats['Sub_Divisions'].progress_map(load_np)

100%|██████████| 50/50 [00:00<00:00, 285.66it/s]


In [106]:
X = salami_beats.head(50).copy()
y = salami_beats.head(50)['Binary_Labels']

## Splits

In [108]:
X_train, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, _, _ = train_test_split(X_train, X_train['Binary_Labels'], test_size=0.2, random_state=42)

In [109]:
# y_train = np.concatenate(X_train['Over_Labels'].values)
y_train = np.concatenate(X_train['Binary_Labels'].values)
y_test = np.concatenate(X_test['Binary_Labels'].values)
y_val = np.concatenate(X_val['Binary_Labels'].values)

ids_test = np.concatenate(X_test['IDS'].values)

# w_train = np.concatenate(X_train['Over_Weights'].values)
w_train = np.concatenate(X_train['Weights'].values)
w_test = np.concatenate(X_test['Weights'].values)
w_val = np.concatenate(X_val['Weights'].values)

# X_train = np.concatenate(X_train['Oversamples'].values)
X_train = np.concatenate(X_train['Sub_Divisions'].values)
X_test = np.concatenate(X_test['Sub_Divisions'].values)
X_val = np.concatenate(X_val['Sub_Divisions'].values)

## Show class imbalance

In [112]:
Counter(y_train)

Counter({1.0: 321, 0.0: 2307})

## Calculate Initial Bias

In [113]:
count = np.bincount(y_train.astype('int64'))
neg, pos = count[0], count[1]
total = neg + pos
initial_bias = np.log([pos/neg])
print(f'Initial bias: {initial_bias}')

Initial bias: [-1.97226214]


## Load model

In [114]:
def build_model(initial_bias):
    initializer = tf.keras.initializers.HeNormal()
    bias_initializer = tf.keras.initializers.Constant(initial_bias)
    initializer = tf.keras.initializers.HeNormal()
    return Sequential([
        Input(shape=(N_MELS, 4, 33)),
        Conv2D(8, 8, activation='relu', kernel_initializer=initializer, padding="same"),
        MaxPooling2D(pool_size=(5, 2)),
        BatchNormalization(),
        Dropout(0.5),
        Conv2D(16, 6, activation='relu', padding="same", kernel_initializer=initializer),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Dropout(0.5),
        Flatten(),
        Dense(128, activation='sigmoid'),
        Dropout(0.5),
        Dense(1, activation='sigmoid', bias_initializer=bias_initializer)
    ])

## Train model

In [116]:
model = build_model(initial_bias)

sgd = tf.keras.optimizers.SGD(lr=0.05, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=get_metrics())

results = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=20, mode='max', restore_best_weights=True)

history = model.fit(
    X_train, 
    y_train, 
    batch_size=128,
    epochs=80, 
    shuffle=True,
    verbose=1,
    sample_weight=w_train,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
) 

Loss: 0.3933
Epoch 1/80
21/21 [==============================] - 3s 101ms/step - loss: 0.7697 - accuracy: 0.2371 - precision: 0.1543 - recall: 0.1807 - auc: 0.5301 - val_loss: 0.3851 - val_accuracy: 0.3644 - val_precision: 0.3333 - val_recall: 0.0575 - val_auc: 0.5702
Epoch 2/80
21/21 [==============================] - 2s 91ms/step - loss: 0.7218 - accuracy: 0.2097 - precision: 0.1703 - recall: 0.1215 - auc: 0.5712 - val_loss: 0.5782 - val_accuracy: 0.1331 - val_precision: 0.1892 - val_recall: 0.3218 - val_auc: 0.6311
Epoch 3/80
21/21 [==============================] - 2s 91ms/step - loss: 0.7131 - accuracy: 0.2013 - precision: 0.2029 - recall: 0.1745 - auc: 0.5811 - val_loss: 0.4071 - val_accuracy: 0.2886 - val_precision: 0.2105 - val_recall: 0.0920 - val_auc: 0.6450
Epoch 4/80
21/21 [==============================] - 2s 90ms/step - loss: 0.7009 - accuracy: 0.2519 - precision: 0.2308 - recall: 0.2336 - auc: 0.6051 - val_loss: 0.3546 - val_accuracy: 0.4502 - val_precision: 0.2941 - val

## Calculate predictions

In [117]:
preds = model.predict(X_test, batch_size=1, verbose=1)
score = model.evaluate(X_test, y_test)

29/29 [==============================] - 0s 7ms/step - loss: 0.4098 - accuracy: 0.4759 - precision: 0.3173 - recall: 0.2750 - auc: 0.7068


## Print classification report

In [118]:
y_pred = [1 if (p > 0.5) else 0 for p in preds]
y_pred = np.asarray(y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90       792
         1.0       0.32      0.28      0.29       120

    accuracy                           0.83       912
   macro avg       0.60      0.59      0.60       912
weighted avg       0.82      0.83      0.82       912



## Calculate F1-Score

In [119]:
f_score, precision, recall = evaluate(salami_beats, preds, ids_test, True)

print("F-score: ", f_score)
print("Precision: ", precision)
print("Recall: ", recall)

F-score:  0.26178511249133835
Precision:  0.2889189976689977
Recall:  0.25780856824335086


## Best score
 * Every iteration of this notebook has a different result
 * The best score achieved is reported below

```
3 seconds:
F-score:  0.7079411816253922
Precision:  0.7491938013677144
Recall:  0.7204761904761905

0.5 seconds:
F-score:  0.3044451156585869
Precision:  0.2801110180142438
Recall:  0.35031746031746025

2 bars:
F-score:  0.7391024180497865
Precision:  0.7825271347010477
Recall:  0.7499206349206349
```

### Best score architecture

```
model = Sequential([
    Input(shape=(N_MELS, 4, 33)),
    Conv2D(8, 8, activation='relu', kernel_initializer=initializer, padding="same"),
    MaxPooling2D(pool_size=(5, 2)),
    BatchNormalization(),
    Dropout(0.5),
    Conv2D(16, 6, activation='relu', padding="same", kernel_initializer=initializer),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='sigmoid'),
    Dropout(0.5),
    Dense(1, activation='sigmoid', bias_initializer=bias_initializer)
])
```

## Save model

## Save results